## Train LSTM model

In this notebook we will train a LSTM model for Sentiment Analysis in English

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
from datasets import Dataset, Value, ClassLabel, Features
from pysentimiento.preprocessing import preprocess_tweet
from pysentimiento.emotion import load_datasets


train_dataset, dev_dataset, test_dataset = load_datasets(preprocessing_args={
    "user_token": "USUARIO",
    "url_token": "URL",
    "hashtag_token": "hashtag",
    "emoji_wrapper": "",
})


In [2]:
from collections import Counter
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
import unidecode

tokenizer = get_tokenizer("spacy", "es_core_news_sm")

counter = Counter()

"""
No overfitting, just taking the embeddings
"""
for dataset in [train_dataset, dev_dataset, test_dataset]:
    for example in dataset:
        tokens = tokenizer(unidecode.unidecode(example["text"].lower()))
        counter.update(tokens)

# Meto todas
vocab = Vocab(counter, min_freq=1)

len(vocab)


16313

In [3]:

def tokenize(batch):
    text = unidecode.unidecode(batch['text'].lower())
    tokens = tokenizer(text)
    token_ids = [vocab.stoi[t] for t in tokens]
    return {"input_ids": token_ids}

batch_size = 32

eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=False)
dev_dataset = dev_dataset.map(tokenize, batched=False)
test_dataset = test_dataset.map(tokenize, batched=False)

In [4]:
def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['label']})
    dataset.set_format(type='torch', columns=['input_ids', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

In [5]:
import torch
import fasttext

fasttext_model = fasttext.load_model("../../embeddings/cc.es.300.bin")


DIM = fasttext_model.get_word_vector("random").shape[0]
emb_matrix = torch.randn(len(vocab), DIM)
UNK_IDX = vocab.stoi["<unk>"]
PAD_IDX = vocab.stoi["<pad>"]

# emb_matrix[UNK_IDX] = 0
emb_matrix[PAD_IDX] = 0

for i, word in enumerate(vocab.itos):
    if i == UNK_IDX or i == PAD_IDX:
        # Let them unmodified
        pass
    else:
        emb_matrix[i] = torch.tensor(fasttext_model.get_word_vector(word))



In [6]:
import torch 
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

PAD_IDX = vocab.stoi["<pad>"]

def collate_batch(batch):
    labels = [t["labels"] for t in batch]
    input_ids = [t["input_ids"] for t in batch]

    # Return text, text_lens, labels
    return pad_sequence(input_ids, padding_value=PAD_IDX, batch_first=True), torch.tensor([len(t) for t in input_ids]), torch.tensor(labels)


train_dataloader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_batch)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, collate_fn=collate_batch)
test_dataset = DataLoader(test_dataset, batch_size=16, collate_fn=collate_batch)


In [8]:
import pytorch_lightning as pl
import torch.nn.functional as F
from pysentimiento.emotion import id2label
from pysentimiento.metrics import get_metrics
from torch import nn

class RNNModel(pl.LightningModule):
    def __init__(self, vocab_size, embedding_dim, pad_idx, rnn_units, num_labels, num_layers=1,
                 bidirectional=False, dropout=0.25, embedding_matrix=None, freeze_embeddings=True):

        super().__init__()

        if embedding_matrix is not None:
            self.embedding = nn.Embedding.from_pretrained(
                embedding_matrix, padding_idx=pad_idx,
                freeze=freeze_embeddings
            )
        else:
            self.embedding = nn.Embedding(
                vocab_size, embedding_dim,
                padding_idx = pad_idx)

        self.rnn = nn.GRU(embedding_dim,
                   rnn_units,
                   num_layers=num_layers,
                   bidirectional=bidirectional, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        factor = 2 if bidirectional else 1

        self.fc = nn.Linear(rnn_units * factor, num_labels)

    def forward(self, text, text_lens):
        #text = [batch_size, text len]
        #permuted = text.permute(1, 0)
        # permuted shape [batch_size, sent len]
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            # WTF no sé por qué hago esto de cpu
            embedded, text_lens.to("cpu"), batch_first=True, enforce_sorted=False)

        packed_output, _ = self.rnn(packed_embedded)
        # hidden is the last state of the

        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        output = self.dropout(output)
        # output is shape [seq, batch, hid]
        s = output.permute(1, 0, 2)
        # now [batch, seq, hid]
        mean = s.sum(dim=1) / text_lens.view(-1, 1)

        return self.fc(mean)

    def training_step(self, batch, batch_idx):
        x, lens, y = batch
        outs = self.forward(x, lens)
        loss = F.cross_entropy(outs, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, lens, y = batch
        outs = self.forward(x, lens)
        loss = F.cross_entropy(outs, y)
        preds = outs.argmax(-1).cpu()
        metrics = get_metrics(preds, y.cpu(), id2label)
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)

        for k, v in metrics.items():
            self.log("val_"+k, v, prog_bar=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, lens, y = batch
        outs = self.forward(x, lens)
        preds = outs.argmax(-1).cpu()
        metrics = get_metrics(preds, y.cpu(), id2label)

        for k, v in metrics.items():
            self.log("test_"+k, v, prog_bar=True, on_epoch=True)

    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = RNNModel(
    vocab_size=len(vocab), embedding_dim=DIM, pad_idx=PAD_IDX, rnn_units=256, embedding_matrix=emb_matrix,
    freeze_embeddings=True, num_labels=7,
)

trainer = pl.Trainer(
    max_epochs=10, 
    gpus=1
)
trainer.fit(model, train_dataloader, dev_dataloader)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 4.9 M 
1 | rnn       | GRU       | 428 K 
2 | dropout   | Dropout   | 0     
3 | fc        | Linear    | 1.8 K 
----------------------------------------
430 K     Trainable params
4.9 M     Non-trainable params
5.3 M     Total params
21.297    Total estimated model params size (MB)


/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  warning_cache.deprecation(


In [10]:
trainer.test(model, test_dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.662492573261261,
 'test_anger_f1': 0.15206381678581238,
 'test_anger_precision': 0.18161116540431976,
 'test_anger_recall': 0.15789207816123962,
 'test_disgust_f1': 0.0,
 'test_disgust_precision': 0.0,
 'test_disgust_recall': 0.0,
 'test_fear_f1': 0.03816338628530502,
 'test_fear_precision': 0.043887894600629807,
 'test_fear_recall': 0.03577817603945732,
 'test_joy_f1': 0.4620981514453888,
 'test_joy_precision': 0.46976178884506226,
 'test_joy_recall': 0.48800334334373474,
 'test_macro_f1': 0.21941280364990234,
 'test_macro_precision': 0.22789530456066132,
 'test_macro_recall': 0.23115865886211395,
 'test_micro_f1': 0.662492573261261,
 'test_others_f1': 0.6519678831100464,
 'test_others_precision': 0.6186308264732361,
 'test_others_recall': 0.7238366007804871,
 'test_sadness_f1': 0.1889805644750595,
 'test_sadness_precision': 0.21458955109119415,
 'test_sadness_rec

[{'test_others_f1': 0.6519678831100464,
  'test_others_precision': 0.6186308264732361,
  'test_others_recall': 0.7238366007804871,
  'test_joy_f1': 0.4620981514453888,
  'test_joy_precision': 0.46976178884506226,
  'test_joy_recall': 0.48800334334373474,
  'test_sadness_f1': 0.1889805644750595,
  'test_sadness_precision': 0.21458955109119415,
  'test_sadness_recall': 0.17888949811458588,
  'test_anger_f1': 0.15206381678581238,
  'test_anger_precision': 0.18161116540431976,
  'test_anger_recall': 0.15789207816123962,
  'test_surprise_f1': 0.04261578246951103,
  'test_surprise_precision': 0.06678592413663864,
  'test_surprise_recall': 0.033710990101099014,
  'test_disgust_f1': 0.0,
  'test_disgust_precision': 0.0,
  'test_disgust_recall': 0.0,
  'test_fear_f1': 0.03816338628530502,
  'test_fear_precision': 0.043887894600629807,
  'test_fear_recall': 0.03577817603945732,
  'test_micro_f1': 0.662492573261261,
  'test_macro_f1': 0.21941280364990234,
  'test_macro_precision': 0.2278953045606

## Twitter Embeddings

In [11]:
import torch
import fasttext

fasttext_model = fasttext.load_model("../../embeddings/tweet_dim_300_ws_5.bin")


DIM = fasttext_model.get_word_vector("random").shape[0]
emb_matrix = torch.randn(len(vocab), DIM)
UNK_IDX = vocab.stoi["<unk>"]
PAD_IDX = vocab.stoi["<pad>"]

# emb_matrix[UNK_IDX] = 0
emb_matrix[PAD_IDX] = 0

for i, word in enumerate(vocab.itos):
    if i == UNK_IDX or i == PAD_IDX:
        # Let them unmodified
        pass
    else:
        emb_matrix[i] = torch.tensor(fasttext_model.get_word_vector(word))



In [12]:
model = RNNModel(
    vocab_size=len(vocab), embedding_dim=DIM, pad_idx=PAD_IDX, rnn_units=256, embedding_matrix=emb_matrix,
    freeze_embeddings=True, num_labels=7,
)

trainer = pl.Trainer(
    max_epochs=10, 
    gpus=1
)
trainer.fit(model, train_dataloader, dev_dataloader)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 4.9 M 
1 | rnn       | GRU       | 428 K 
2 | dropout   | Dropout   | 0     
3 | fc        | Linear    | 1.8 K 
----------------------------------------
430 K     Trainable params
4.9 M     Non-trainable params
5.3 M     Total params
21.297    Total estimated model params size (MB)


/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


In [13]:
trainer.test(model, test_dataset)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6702444553375244,
 'test_anger_f1': 0.13744033873081207,
 'test_anger_precision': 0.16878779232501984,
 'test_anger_recall': 0.12739886343479156,
 'test_disgust_f1': 0.009540846571326256,
 'test_disgust_precision': 0.009540846571326256,
 'test_disgust_recall': 0.009540846571326256,
 'test_fear_f1': 0.07155635207891464,
 'test_fear_precision': 0.07473663240671158,
 'test_fear_recall': 0.07632677257061005,
 'test_joy_f1': 0.4897812604904175,
 'test_joy_precision': 0.4881468415260315,
 'test_joy_recall': 0.5282502174377441,
 'test_macro_f1': 0.2342625856399536,
 'test_macro_precision': 0.24061787128448486,
 'test_macro_recall': 0.24590569734573364,
 'test_micro_f1': 0.6702444553375244,
 'test_others_f1': 0.650103747844696,
 'test_others_precision': 0.6217681765556335,
 'test_others_recall': 0.7159823179244995,
 'test_sadness_f1': 0.21749188005924225,
 'test_sadness_pr

[{'test_others_f1': 0.650103747844696,
  'test_others_precision': 0.6217681765556335,
  'test_others_recall': 0.7159823179244995,
  'test_joy_f1': 0.4897812604904175,
  'test_joy_precision': 0.4881468415260315,
  'test_joy_recall': 0.5282502174377441,
  'test_sadness_f1': 0.21749188005924225,
  'test_sadness_precision': 0.241837739944458,
  'test_sadness_recall': 0.205959752202034,
  'test_anger_f1': 0.13744033873081207,
  'test_anger_precision': 0.16878779232501984,
  'test_anger_recall': 0.12739886343479156,
  'test_surprise_f1': 0.0639236718416214,
  'test_surprise_precision': 0.07950706034898758,
  'test_surprise_recall': 0.05788113549351692,
  'test_disgust_f1': 0.009540846571326256,
  'test_disgust_precision': 0.009540846571326256,
  'test_disgust_recall': 0.009540846571326256,
  'test_fear_f1': 0.07155635207891464,
  'test_fear_precision': 0.07473663240671158,
  'test_fear_recall': 0.07632677257061005,
  'test_micro_f1': 0.6702444553375244,
  'test_macro_f1': 0.2342625856399536,